In [231]:
class MapDBackendRenderer:
    def __init__(self, connection, data):
        self.connection = connection
        self.data = data
    
    def _repr_mimebundle_(self, include=None, exclude=None):
        bundle = {
            'connection': self.connection,
            'vega': self.data
        }
        return {
            'application/vnd.mapd.vega+json': bundle
        }
    

In [232]:
from IPython.core.magic import register_cell_magic
from IPython.display import display
from ipykernel import jsonutil
import yaml
import ast

@register_cell_magic
def mapd(line, cell):
    connection_data = ast.literal_eval(line)
    vega = yaml.load(cell)
    display(MapDBackendRenderer(connection_data, vega))
    

In [233]:
import pymapd
connection_data = dict(
    user='mapd',
    password='HyperInteractive',
    host='metis.mapd.com',
    port='443',
    dbname='mapd',
    protocol='https'
)
con = pymapd.connect(**connection_data)

In [240]:
%%mapd $connection_data

width: 384
height: 564
data:
  - name: 'tweets'
    sql: 'SELECT goog_x as x, goog_y as y, tweets_nov_feb.rowid FROM tweets_nov_feb' 

scales:
  - name: 'x'
    type: 'linear'
    domain:
      - -3650484.1235206556
      -  7413325.514451755
    range: 'width'
  - name: 'y'
    type: 'linear'
    domain:
      - -5778161.9183506705
      -  10471808.487466192
    range: 'height'
marks:
  - type: 'points'
    from:
      data: 'tweets'
    properties:
      x:
        scale: 'x'
        field: 'x'
      y:
        scale: 'y'
        field: 'y'
      fillColor: 'green'
      size:
        value: 1      